# Prepare GCAM Data

This notebook collects and organizes the total generation (MWh) by technology in 2050 under the Clean Grid and Business-as-usual Scenarios

### Download Required Dataset

#### GCAM-USA Capacity Expansion Plan under Net Zero and Business-as-Usual Scenarios

**Dataset Title:** GCAM-USA Scenarios for GODEEEP

**Description from source:** This dataset contains a set of twelve future (2020-2050) scenarios modeled by GCAM-USA for the GODEEEP project for the purpose of studying the effects of climate, socioeconomic change, technology change, current decarbonization incentives, and longer-term decarbonization policies on the U.S. energy-economy, the electricity grid, human well-being, and the environment.

Download the GCAM-USA dataset from here: https://doi.org/10.5281/zenodo.10642507

**Reference:**
> Ou, Y., Zhang, Y., Waldhoff, S., & Iyer, G. (2024). GCAM-USA Scenarios for GODEEEP (v3.0.2) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.10642507

______________

## Steps:

1. Download and extract the GCAM-USA dataset inside the `/data/input_data/gcam_data` directory of this repository as the paths in this notebook are set to that expectation.
2. Run the scripts below
3. Output file will be saved to `/data/output_data/generation_data`

### Imports

In [7]:
import gcamreader
import numpy as np
import pandas as pd
import os

### Data Paths

In [75]:
# set year of analysis
year = 2050

# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data')

# gcam data dir
gcam_data_dir = os.path.join(data_dir, 'gcam_data')

# bau gcam database
#gcam_db_path = os.path.join(gcam_data_dir, 'GODEEEP_GCAM-USA_Pathways')

# # bau file
# bau_gcam_db_file = 'Main_database_basexdb_BAU_Climate'

# # net zero file
# nz_gcam_db_file = 'Main_database_basexdb_NetZeroNoCCS_Climate'

# bau file
#bau_gcam_db_file = 'bau_ira_ccs_climate'

bau_gcam_db_file = 'godeeep_generation_bau_ira_ccs_climate.xlsx'

# net zero file
#nz_gcam_db_file = 'nz_ira_ccs_climate'
nz_gcam_db_file = 'godeeep_generation_nz_ira_ccs_climate.xlsx'

# gcam query path
gcam_query_path = os.path.join(data_dir, 'gcam_query_xlm', 'subSetQueries.xml')

# output data dir
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data', 'processed_generation_data')

# output file path
output_path = os.path.join(output_dir, f'gcam_generation_state_tech_{year}.csv')

# query name for the generation data
generation_query_name = "elec gen by gen tech cogen USA"

### Settings

In [41]:
STATE_ABBREVIATIONS_TO_NAMES = {
    'AB': 'alberta', 'AK': 'alaska', 'AL': 'alabama', 'AR': 'arkansas', 'AZ': 'arizona',
    'CA': 'california', 'BC': 'british_columbia', 'MX': 'mexico',
    'CO': 'colorado', 'CT': 'connecticut', 'DC': 'district_of_columbia', 'DE': 'delaware',
    'FL': 'florida', 'GA': 'georgia', 'HI': 'hawaii', 'IA': 'iowa', 'ID': 'idaho', 'IL': 'illinois',
    'IN': 'indiana', 'KS': 'kansas', 'KY': 'kentucky', 'LA': 'louisiana', 'MA': 'massachusetts',
    'MD': 'maryland', 'ME': 'maine', 'MI': 'michigan', 'MN': 'minnesota', 'MO': 'missouri',
    'MS': 'mississippi', 'MT': 'montana', 'NC': 'north_carolina', 'ND': 'north_dakota',
    'NE': 'nebraska', 'NH': 'new_hampshire', 'NJ': 'new_jersey', 'NM': 'new_mexico', 'NV': 'nevada',
    'NY': 'new_york', 'OH': 'ohio', 'OK': 'oklahoma', 'OR': 'oregon', 'PA': 'pennsylvania',
    'RI': 'rhode_island', 'SC': 'south_carolina', 'SD': 'south_dakota', 'TN': 'tennessee',
    'TX': 'texas', 'UT': 'utah', 'VA': 'virginia', 'VT': 'vermont', 'WA': 'washington',
    'WI': 'wisconsin', 'WV': 'west_virginia', 'WY': 'wyoming'
}

# western interconnection states

WECC = {'AZ': 'arizona', 'CA': 'california', 'CO': 'colorado', 'ID': 'idaho',  'MT': 'montana', 
        'NM': 'new_mexico', 'NV': 'nevada', 'UT':'utah', 'OR': 'oregon', 'WA': 'washington','WY': 'wyoming'}

In [101]:
# Convert native gcam Exajoule output to terawatt-hours
EXAJOULES_TO_MWH = 277.77777777778 * 1000

EXAJOULES_TO_KWH = 277777777777.7778

### Read in GCAM Databases

In [80]:
# dictionary of technology types to collect
renewable_dict = {'wind': ['wind_base', 'wind_base_storage','wind_int', 'wind_subpeak'],
                  'solar': ['PV_base_storage', 'PV_int','PV_peak','PV_subpeak']}

### Net Zero

In [102]:
# read in file
nz_generation = pd.read_excel(os.path.join(gcam_data_dir, nz_gcam_db_file))

# reduce to western interconnection states
nz_generation = nz_generation[nz_generation.region.isin(WECC.keys())]

# collect technologies of interest
nz_generation = nz_generation[nz_generation.subsector.isin(['solar', 'wind'])]
nz_generation = nz_generation[(nz_generation.technology.isin(renewable_dict['wind'])) | (nz_generation.technology.isin(renewable_dict['solar']))]

# simplify naming and columns to include
nz_generation['tech_type'] = np.where((nz_generation.technology.isin(renewable_dict['wind'])), 'Wind', 'Solar PV') 
nz_generation = nz_generation[['region', 2050, 'tech_type']]

# group data by technology type
nz_generation = nz_generation.groupby(['region','tech_type'], as_index=False).sum()

# convert generation to TWh
nz_generation[2050] = round(nz_generation[2050] * EXAJOULES_TO_KWH, 0)

# set scenario name
nz_generation['scenario'] = 'net_zero_ira_ccs_climate'
nz_generation


,region,tech_type,2050,scenario
0,AZ,Solar PV,4.804434e+10,net_zero_ira_ccs_climate
1,AZ,Wind,2.729310e+10,net_zero_ira_ccs_climate
2,CA,Solar PV,1.144534e+11,net_zero_ira_ccs_climate
3,CA,Wind,1.201025e+11,net_zero_ira_ccs_climate
4,CO,Solar PV,4.861935e+10,net_zero_ira_ccs_climate
5,CO,Wind,6.955589e+10,net_zero_ira_ccs_climate
6,ID,Solar PV,1.881205e+09,net_zero_ira_ccs_climate
7,ID,Wind,1.239708e+10,net_zero_ira_ccs_climate
8,MT,Solar PV,1.647290e+10,net_zero_ira_ccs_climate
9,MT,Wind,1.283926e+11,net_zero_ira_ccs_climate


## BAU

In [103]:
# read in file
bau_generation = pd.read_excel(os.path.join(gcam_data_dir, bau_gcam_db_file))

# reduce to western interconnection states
bau_generation = bau_generation[bau_generation.region.isin(WECC.keys())]

# collect technologies of interest
bau_generation = bau_generation[bau_generation.subsector.isin(['solar', 'wind'])]
bau_generation = bau_generation[(bau_generation.technology.isin(renewable_dict['wind'])) | (bau_generation.technology.isin(renewable_dict['solar']))]

# simplify naming and columns to include
bau_generation['tech_type'] = np.where((bau_generation.technology.isin(renewable_dict['wind'])), 'Wind', 'Solar PV') 
bau_generation = bau_generation[['region', 2050, 'tech_type']]

# group data by technology type
bau_generation = bau_generation.groupby(['region','tech_type'], as_index=False).sum()

# convert generation to TWh
bau_generation[2050] = round(bau_generation[2050] * EXAJOULES_TO_KWH, 0)

bau_generation['scenario'] = 'business_as_usual_ira_ccs_climate'


## Combine output

In [104]:
# combine bau and net zero files
gcam_data = pd.concat([bau_generation, nz_generation])

# rename generation column
gcam_data = gcam_data.rename(columns= {2050:'2050_KWh'})

# collect full state names
gcam_data['region_name'] = gcam_data['region'].map(WECC)

gcam_data

,region,tech_type,2050_KWh,scenario,region_name
0,AZ,Solar PV,3.814787e+10,business_as_usual_ira_ccs_climate,arizona
1,AZ,Wind,2.321585e+10,business_as_usual_ira_ccs_climate,arizona
2,CA,Solar PV,7.066141e+10,business_as_usual_ira_ccs_climate,california
3,CA,Wind,7.591400e+10,business_as_usual_ira_ccs_climate,california
4,CO,Solar PV,3.474630e+10,business_as_usual_ira_ccs_climate,colorado
5,CO,Wind,4.565293e+10,business_as_usual_ira_ccs_climate,colorado
6,ID,Solar PV,1.428787e+09,business_as_usual_ira_ccs_climate,idaho
7,ID,Wind,1.105339e+10,business_as_usual_ira_ccs_climate,idaho
8,MT,Solar PV,9.418810e+09,business_as_usual_ira_ccs_climate,montana
9,MT,Wind,8.857231e+10,business_as_usual_ira_ccs_climate,montana


#### Save to file

In [105]:
gcam_data.to_csv(output_path, index=False)

### Functions

In [10]:
def get_query_by_name(queries, name):
    """Return query for given name"""
    return next((x for x in queries if x.title == name), None)

### Connect and Process GCAM DATA

##### Step 1. Connect to the GCAM database

In [11]:
# net zero
nz_conn = gcamreader.LocalDBConn(gcam_data_dir, nz_gcam_db_file)

# business-as-usual
bau_conn = gcamreader.LocalDBConn(gcam_data_dir, bau_gcam_db_file)

Database scenarios: R_02b_NZ_climate
Database scenarios: R_01b_BAU_climate


##### Step 2. Create a list of queries

In [33]:
# list of queries\
queries = gcamreader.parse_batch_query(gcam_query_path)
queries

##### Step 3. Collect the generation data

In [36]:
generation_query_name = "elec gen by gen tech USA"

# net zero generation data
nz_generation = nz_conn.runQuery(get_query_by_name(queries, generation_query_name))

# bau generation data
#bau_generation = bau_conn.runQuery(get_query_by_name(queries, generation_query_name))

Model interface run failed.
Command line: 
	java -cp /Users/mong275/.pyenv/versions/3.11.0/envs/visualization_3.11/lib/python3.11/site-packages/gcamreader/ModelInterface/jars/*:/Users/mong275/.pyenv/versions/3.11.0/envs/visualization_3.11/lib/python3.11/site-packages/gcamreader/ModelInterface/ModelInterface.jar -Xmx4g -Dorg.basex.DBPATH=/Users/mong275/repos/mongird-etal_2024_tbd/data/input_data/gcam_data -DModelInterface.SUPPRESS_OUTPUT=True org.basex.BaseX -smethod=csv -scsv=header=yes,format=xquery -i nz_ira_ccs_climate RUN /var/folders/zp/6_04pmq97nq_cq_837pfbx200000gn/T/tmp3kkzvf17
Query string: 
	<supplyDemandQuery title="elec gen by gen tech USA">
   <axis1 name="technology">technology</axis1>
   <axis2 name="Year">physical-output[@vintage]</axis2>
   <xPath buildList="true" dataName="output" group="false" sumAll="false">*[@type='sector' and (@name='electricity' or 
                @name='base load generation' or @name='intermediate generation' or @name='subpeak generation' or @n

CalledProcessError: Command '['java', '-cp', '/Users/mong275/.pyenv/versions/3.11.0/envs/visualization_3.11/lib/python3.11/site-packages/gcamreader/ModelInterface/jars/*:/Users/mong275/.pyenv/versions/3.11.0/envs/visualization_3.11/lib/python3.11/site-packages/gcamreader/ModelInterface/ModelInterface.jar', '-Xmx4g', '-Dorg.basex.DBPATH=/Users/mong275/repos/mongird-etal_2024_tbd/data/input_data/gcam_data', '-DModelInterface.SUPPRESS_OUTPUT=True', 'org.basex.BaseX', '-smethod=csv', '-scsv=header=yes,format=xquery', '-i', 'nz_ira_ccs_climate', 'RUN', '/var/folders/zp/6_04pmq97nq_cq_837pfbx200000gn/T/tmp3kkzvf17']' returned non-zero exit status 1.

##### Step 4. Process Data

In [ ]:
EXAJOULES_TO_GIGAWATTS = 277.77777777778 * 1000 / 8760

###### Net Zero

In [35]:
nz_generation

,Units,scenario,region,sector,subsector,technology,Year,value
0,EJ,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Africa_Eastern,elect_td_bld,rooftop_pv,"rooftop_pv,year=2020",2020,0.034789
1,EJ,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Africa_Eastern,elect_td_bld,rooftop_pv,"rooftop_pv,year=2025",2025,0.072080
2,EJ,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Africa_Eastern,elect_td_bld,rooftop_pv,"rooftop_pv,year=2030",2030,0.174199
3,EJ,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Africa_Eastern,elect_td_bld,rooftop_pv,"rooftop_pv,year=2035",2035,0.337721
4,EJ,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Africa_Eastern,elect_td_bld,rooftop_pv,"rooftop_pv,year=2040",2040,0.580254
...,...,...,...,...,...,...,...,...
55415,km^3,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Middle East,electricity,refined liquids,"refined liquids (steam/CT) (seawater),year=2015",2030,0.159889
55416,km^3,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Middle East,electricity,refined liquids,"refined liquids (steam/CT) (seawater),year=2015",2035,0.067993
55417,km^3,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Middle East,electricity,refined liquids,"refined liquids (steam/CT) (seawater),year=2015",2040,0.029492
55418,km^3,"R_02b_NZ_climate,date=2023-11-10T11:38:14-07:00",Middle East,electricity,refined liquids,"refined liquids (steam/CT) (seawater),year=2015",2045,0.009577


In [32]:
nz_generation.region.unique()

array(['Africa_Eastern', 'Africa_Northern', 'Africa_Southern',
       'Africa_Western', 'Alaska grid', 'Argentina', 'Australia_NZ',
       'Brazil', 'California grid', 'Canada',
       'Central America and Caribbean', 'Central Asia',
       'Central East grid', 'Central Northeast grid',
       'Central Northwest grid', 'Central Southwest grid', 'China',
       'Colombia', 'EU-12', 'EU-15', 'Europe_Eastern', 'Europe_Non_EU',
       'European Free Trade Association', 'Florida grid', 'Hawaii grid',
       'India', 'Indonesia', 'Japan', 'Mexico', 'Mid-Atlantic grid',
       'Middle East', 'New England grid', 'New York grid',
       'Northwest grid', 'Pakistan', 'Russia', 'South Africa',
       'South America_Northern', 'South America_Southern', 'South Asia',
       'South Korea', 'Southeast Asia', 'Southeast grid',
       'Southwest grid', 'Taiwan', 'Texas grid'], dtype=object)

In [ ]:
generation = nz_gneration

generation['vintage'] = generation['technology'].str.slice(-4).astype(int)
generation['technology'] = generation['technology'].str.slice(0, -10)
generation['subsector.1'] = generation['subsector.1'].str.slice(0, -8)

# convert EJ to GW
generation['value'] = generation['value'] * EXAJOULES_TO_GIGAWATTS
generation = generation.drop(columns=['subsector']).rename(columns={
                        'Year': 'year',
                        'Units': 'units',
                        'value': 'generation_GW',
                        'subsector.1': 'subsector',
                    })
generation = generation[['year', 'vintage', 'region', 'sector', 'subsector', 'technology', 'generation_GW']].reset_index(drop=True)